In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from pathlib import Path
from skimage.io import imread
import json
%matplotlib qt5

## Reading emitter location pixels from the first set of data

In [3]:

bead_data_200 = []
for pos in range(1, 6):
    bead_locations = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-23-CA3091/beadsLocalizationAccuracy/bead_locations/')
    images_dir = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-23-CA3091/beadsLocalizationAccuracy/lowIntensity/')
    bead_data_path = bead_locations / Path('Pos' + str(pos))
    with open(bead_data_path) as fh:
        all_beads = json.load(fh)
        if len(all_beads) != 0:
            for one_bead in all_beads:
                bead_data_200.append({'path': images_dir / Path('Pos' + str(pos).zfill(2)) / Path('Pos0'), 'x': one_bead['x'][0], 'y': one_bead['y'][0]})
            #print(len(all_beads))

bead_data_50 = []
for pos in range(100, 150):
    bead_locations = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4713/fluorChannelRegistration3D/bead_positions/')
    images_dir = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4713/fluorChannelRegistration3D/zStack_locError_101/')
    bead_data_path = bead_locations / Path('Pos' + str(pos))
    with open(bead_data_path) as fh:
        all_beads = json.load(fh)
        if len(all_beads) != 0:
            for one_bead in all_beads:
                bead_data_50.append({'path': images_dir / Path('Pos' + str(pos)), 'x': one_bead['x'][0], 'y': one_bead['y'][0]})
            #print(len(all_beads))


for pos in range(100, 150):
    bead_locations = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4719/fluorChannelRegistration3D/bead_positions_101/')
    images_dir = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4719/fluorChannelRegistration3D/zStack_locError_101/')
    bead_data_path = bead_locations / Path('Pos' + str(pos))
    if bead_data_path.exists():
        with open(bead_data_path) as fh:
            all_beads = json.load(fh)
            if len(all_beads) != 0:
                for one_bead in all_beads:
                    bead_data_50.append({'path': images_dir / Path('Pos' + str(pos)), 'x': one_bead['x'][0], 'y': one_bead['y'][0]})
                #print(len(all_beads))


for pos in range(100, 150):
    bead_locations = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4719/fluorChannelRegistration3D/bead_positions_102/')
    images_dir = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4719/fluorChannelRegistration3D/zStack_locError_102/')
    bead_data_path = bead_locations / Path('Pos' + str(pos))
    if bead_data_path.exists():
        with open(bead_data_path) as fh:
            all_beads = json.load(fh)
            if len(all_beads) != 0:
                for one_bead in all_beads:
                    bead_data_50.append({'path': images_dir / Path('Pos' + str(pos)), 'x': one_bead['x'][0], 'y': one_bead['y'][0]})
                #print(len(all_beads))


for pos in range(100, 150):
    bead_locations = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4719/fluorChannelRegistration3D/bead_positions_103/')
    images_dir = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4719/fluorChannelRegistration3D/zStack_locError_103/')
    bead_data_path = bead_locations / Path('Pos' + str(pos))
    if bead_data_path.exists():
        with open(bead_data_path) as fh:
            all_beads = json.load(fh)
            if len(all_beads) != 0:
                for one_bead in all_beads:
                    bead_data_50.append({'path': images_dir / Path('Pos' + str(pos)), 'x': one_bead['x'][0], 'y': one_bead['y'][0]})
                #print(len(all_beads))

In [4]:
len(bead_data_200)

72

In [5]:
len(bead_data_50)

1629

### Load all emitters and place them on a cell segmentation mask


In [6]:
import edt
import pickle
import torch

In [7]:
def map_masks(dists, map_values):
    dists_copy = np.copy(dists)
    for edt_val, value in map_values.items():
        dists_copy[dists == edt_val] = value
    return dists_copy


In [8]:
segmentation_mask = Path('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/EXP-23-CA3045/pool_3045_ter_pos01/phase_venus_mask/Pos01_img_000000000.png')
segmentation_mask = imread(segmentation_mask)

bg_noise_path = Path('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/EXP-23-CA3045/edt_noise.pkl')
with open(bg_noise_path,'rb') as fp:
    edt_noise_map = pickle.load(fp)

In [9]:
def sample_new_cellbg(segmentation_mask, edt_noise_map):
    dists = edt.edt(segmentation_mask)
    dists = np.round(dists)
    dists = np.clip(dists, 0, 6)
    
    cells_bg_alpha = map_masks(dists, edt_noise_map['alphas'])
    cells_bg_beta = map_masks(dists, edt_noise_map['betas'])
    alpha_t = torch.from_numpy(cells_bg_alpha)
    beta_t = 1.0 / torch.from_numpy(cells_bg_beta)
    cell_bg_sampled = torch.distributions.gamma.Gamma(concentration=alpha_t, rate=beta_t).sample()
    return cell_bg_sampled.numpy()

In [10]:
plt.imshow(sample_new_cellbg(segmentation_mask, edt_noise_map))

### Sample and generate image stacks and write to disk

sample one emitter per cell to make tracking easy

In [11]:
np.sum(segmentation_mask != 0)

118708

In [12]:
def generate_one_location_per_cell(segmentation_mask, number_of_points):
    r = []
    c = []
    for i in range(1, number_of_points + 1):
        (x, y) = np.where(segmentation_mask == i*2)
        # pick a random pixel
        chosen_pixel = np.random.choice(np.arange(len(x)), size=1)
        r.append(x[chosen_pixel][0])
        c.append(y[chosen_pixel][0])
    return r, c

In [13]:
r, c = generate_one_location_per_cell(segmentation_mask, 72)

In [14]:
len(r), len(c)

(72, 72)

#### Loop over and overlay on the locations and generate stack

In [15]:
save_images_dir_200 = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/synthetic_data/images200/')

In [23]:
generated_images = [sample_new_cellbg(segmentation_mask, edt_noise_map) for i in range(200)]
#generated_images = [np.zeros(segmentation_mask.shape) for i in range(200)]

In [24]:
generated_images = np.stack(generated_images)

In [25]:
for i, bead in enumerate(bead_data_200):
    stack_path = bead['path']
    print(stack_path)
    img_filenames = sorted(list(stack_path.glob('*.tif')))
    images = []
    for filename in img_filenames:
        images.append(imread(filename))
    images = np.stack(images)[:, 1041:, :]
    # grab the slice from the stack with center being the x, y in the bead dictionary
    bead_x, bead_y = int(bead['y']/65.0), int(bead['x']/65.0)
    #crop roi 21x21
    ROI = 41
    try:
        bead_sliced_from_stack = images[:, bead_x - (ROI//2) : bead_x + (ROI//2) + 1, bead_y - (ROI//2) : bead_y + (ROI//2) + 1]
        print(bead_sliced_from_stack.shape, bead_x, bead_y, r[i], c[i])
        generated_images[:, r[i] - (ROI//2) : r[i] + (ROI//2) + 1, c[i] - (ROI//2) : c[i] + (ROI//2) + 1] = bead_sliced_from_stack
    except Exception as e:
        print(f"Oops bead {i} failed due to {e}")

    print(f"Bead: {i} done")

/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-23-CA3091/beadsLocalizationAccuracy/lowIntensity/Pos01/Pos0
(200, 41, 41) 163 608 116 494
Bead: 0 done
/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-23-CA3091/beadsLocalizationAccuracy/lowIntensity/Pos01/Pos0
(200, 41, 41) 558 438 120 560
Bead: 1 done
/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-23-CA3091/beadsLocalizationAccuracy/lowIntensity/Pos01/Pos0
(200, 41, 41) 588 970 122 659
Bead: 2 done
/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-23-CA3091/beadsLocalizationAccuracy/lowIntensity/Pos01/Pos0
(200, 41, 41) 854 399 127 716
Bead: 3 done
/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-23-CA3091/beadsLocalizationAccuracy/lowIntensity/Pos02/Pos0
(200, 41, 41) 348 285 126 791
Bead: 4 done
/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-23-CA3091/beadsLocalizationAccuracy/lowIntensity/Pos02/Pos0
(200, 41, 41) 269 442 130 857
Bead: 5 done
/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-23-CA3091/beadsLocalizationAccuracy/lowIntensity/Pos02/Pos0


In [26]:
1 + 1

2

In [30]:
plt.figure()
plt.imshow(generated_images[100], cmap='gray')
plt.show()

In [28]:
from skimage.io import imsave

In [29]:
imsave(save_images_dir_200/Path('images_200.tiff'),  generated_images, check_contrast=False, plugin='tifffile')

#### Save 50 image stacks

In [36]:
save_images_dir_50 = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/synthetic_data/images50/')

In [37]:
def generate_one_location_per_cell(segmentation_mask, number_of_points):
    r = []
    c = []
    for i in range(1, number_of_points + 1):
        (x, y) = np.where(segmentation_mask == i)
        # pick a random pixel
        chosen_pixel = np.random.choice(np.arange(len(x)), size=1)
        r.append(x[chosen_pixel][0])
        c.append(y[chosen_pixel][0])
    return r, c

In [38]:

for start in [0, 300, 600, 900, 1200]:
    generated_images = [sample_new_cellbg(segmentation_mask, edt_noise_map) for i in range(50)]
    generated_images = np.stack(generated_images)
    r, c = generate_one_location_per_cell(segmentation_mask, number_of_points=300)
    for i in range(start, start + 300):
        bead = bead_data_50[i]
        stack_path = bead['path']
        print(stack_path)
        img_filenames = sorted(list(stack_path.glob('*.tif')))
        images = []
        for filename in img_filenames:
            images.append(imread(filename))
        images = np.stack(images)[:, 1041:, :]
    
        # grab the slice from the stack with center being the x, y in the bead dictionary
        bead_x, bead_y = int(bead['x']/65.0), int(bead['y']/65.0)
        #crop roi 
        ROI = 27
        try:
            bead_sliced_from_stack = images[:, bead_x - (ROI//2) : bead_x + (ROI//2) + 1, bead_y - (ROI//2) : bead_y + (ROI//2) + 1]
            print(bead_sliced_from_stack.shape, bead_x, bead_y, r[i-start], c[i-start])
            generated_images[:, r[i-start] - (ROI//2) : r[i-start] + (ROI//2) + 1, c[i-start] - (ROI//2) : c[i-start] + (ROI//2) + 1] = bead_sliced_from_stack
        except Exception as e:
            print(f"Oops bead {i} failed due to {e}")
    
        print(f"Bead: {i} done")
    imsave(save_images_dir_50/Path('images_50_' + str(start) + '.tiff'),  generated_images, check_contrast=False, plugin='tifffile')



/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4713/fluorChannelRegistration3D/zStack_locError_101/Pos100
(50, 27, 27) 425 1118 115 431
Bead: 0 done
/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4713/fluorChannelRegistration3D/zStack_locError_101/Pos101
(50, 27, 27) 341 708 111 481
Bead: 1 done
/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4713/fluorChannelRegistration3D/zStack_locError_101/Pos101
(50, 27, 27) 426 1118 116 534
Bead: 2 done
/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4713/fluorChannelRegistration3D/zStack_locError_101/Pos101
(50, 27, 27) 779 1178 116 560
Bead: 3 done
/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4713/fluorChannelRegistration3D/zStack_locError_101/Pos102
(50, 27, 27) 341 708 118 589
Bead: 4 done
/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4713/fluorChannelRegistration3D/zStack_locError_101/Pos102
(50, 27, 27) 425 1118 117 634
Bead: 5 done
/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4713/fluorChannelRegistration3D/z

In [ ]:
len(r), len(c)